<a href="https://colab.research.google.com/github/Marcin19721205/WSBNeuronowe/blob/main/SiecLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Stworzenie i wizualizacja obiektu

In [19]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# ============================================================
# PARAMETRY SIMULACJI / OBIEKTU
# ============================================================
N           = 5000     # liczba kroków czasowych
dt          = 0.1      # krok czasowy [s]
t           = np.arange(N) * dt

K_proc      = 1.0       # wzmocnienie obiektu
zeta        = 0.4       # współczynnik tłumienia (0<zeta<1 → tłumienie)
omega_n     = 0.15      # częstość naturalna [rad/s] (~1/T)
T_dom       = 1.0 / omega_n   # umowna "stała czasowa" dla opóźnienia

delay_ratio = 1.8       # ułamek stałej czasowej
delay_steps = max(1, int(delay_ratio * T_dom / dt))

print(f"T_dom ≈ {T_dom:.3f} s")
print(f"Delay steps = {delay_steps} (≈ {delay_ratio*100:.1f}% T_dom)")

# ============================================================
# RNG + SZUM: persystentny, skośny, z outlierami
# ============================================================
rng = np.random.default_rng(42)

def generate_persistent_skewed_noise(N,
                                     phi=0.9,
                                     base_sigma=0.3,
                                     skew_factor=0.6,
                                     p_out=0.01,
                                     out_scale=4.0):
    eps  = rng.normal(0.0, base_sigma, size=N)
    noise = np.zeros(N)
    for k in range(1, N):
        noise[k] = phi * noise[k-1] + eps[k]

    skew_part = np.exp(skew_factor * noise)
    skew_part = skew_part - np.mean(skew_part)
    noise_skewed = noise + 0.7 * skew_part

    mask_out = rng.random(N) < p_out
    jumps    = rng.normal(0, out_scale * base_sigma, size=N)
    noise_skewed[mask_out] += jumps[mask_out]

    return noise_skewed

noise = generate_persistent_skewed_noise(N)

# ============================================================
# SYGNAŁ STERUJĄCY MV:
#  - start z MV = 0
#  - skok 0 -> 50% i trzymanie przez ~6*T_dom (faza STEP)
#  - potem PRBS skalowany do T_dom (faza PRBS)
# ============================================================
MV = np.zeros(N)

# czas ustalania: 6 * T_dom (w próbkach)
settle_steps = int(6 * T_dom / dt)
settle_steps = min(settle_steps, N)

# pierwszy skok 0 -> 50%
if settle_steps > 1:
    MV[0] = 0.0
    MV[1:settle_steps] = 50.0
else:
    MV[:] = 50.0

# parametry PRBS w krotnościach T_dom
prbs_min_T_factor = 0.5
prbs_max_T_factor = 2.0

prbs_min_len = max(1, int(prbs_min_T_factor * T_dom / dt))
prbs_max_len = max(prbs_min_len + 1, int(prbs_max_T_factor * T_dom / dt))

prbs_low     = 10.0
prbs_high    = 90.0

current_level = prbs_high
k = settle_steps

while k < N:
    seg_len = int(rng.integers(prbs_min_len, prbs_max_len + 1))
    end_idx = min(N, k + seg_len)
    MV[k:end_idx] = current_level
    current_level = prbs_low if current_level == prbs_high else prbs_high
    k = end_idx

print(f"PRBS segment length in samples: [{prbs_min_len}, {prbs_max_len}]")
print(f"PRBS segment length in seconds: [{prbs_min_len*dt:.2f}, {prbs_max_len*dt:.2f}]")

# ============================================================
# MODEL 2 RZĘDU Z TŁUMIENIEM + OPÓŹNIENIE
# ============================================================
x1 = np.zeros(N)  # CV
x2 = np.zeros(N)  # pochodna

for k in range(1, N):
    if k >= delay_steps:
        u_delayed = MV[k - delay_steps]
    else:
        u_delayed = MV[0]

    dx1 = x2[k-1]
    dx2 = -2.0 * zeta * omega_n * x2[k-1] - (omega_n**2) * x1[k-1] \
          + K_proc * (omega_n**2) * u_delayed

    x1[k] = x1[k-1] + dt * dx1
    x2[k] = x2[k-1] + dt * dx2

CV_clean = x1
CV = CV_clean + noise

# ============================================================
# FAZA TESTU: STEP / PRBS
# ============================================================
phase = np.empty(N, dtype=object)
phase[:settle_steps] = "STEP"
phase[settle_steps:] = "PRBS"

# można też mieć wersję numeryczną, jeśli chcesz pod ML:
# phase_id = np.zeros(N, dtype=int)
# phase_id[settle_steps:] = 1

# ============================================================
# DATAFRAME
# ============================================================
df = pd.DataFrame({
    "t":        t,
    "MV":       MV,
    "CV":       CV,
    "CV_clean": CV_clean,
    "noise":    noise,
    "phase":    phase
    # "phase_id": phase_id,   # jeśli jednak wolisz int
})

print(df.head())
print(df["phase"].value_counts())

# ============================================================
# WIZUALIZACJA
# ============================================================
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=df["t"], y=df["MV"],
                          mode="lines", name="MV (sterowanie)"))
fig1.add_trace(go.Scatter(x=df["t"], y=df["CV"],
                          mode="lines", name="CV (z szumem)"))
fig1.add_trace(go.Scatter(x=df["t"], y=df["CV_clean"],
                          mode="lines", name="CV_clean", line=dict(dash="dot")))

# pionowa linia w miejscu przejścia STEP -> PRBS
fig1.add_vline(x=settle_steps * dt,
               line=dict(dash="dash"),
               annotation_text="START PRBS",
               annotation_position="top right")

fig1.update_layout(
    title="Obiekt 2 rzędu z opóźnieniem + szum (STEP + PRBS, z fazą)",
    xaxis_title="t [s]",
    yaxis_title="Wartość",
)
fig1.show()

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=df["t"], y=df["noise"],
                          mode="lines", name="noise"))
fig2.update_layout(
    title="Szum (persystentny, skośny, z outlierami)",
    xaxis_title="t [s]",
    yaxis_title="noise"
)
fig2.show()


T_dom ≈ 6.667 s
Delay steps = 120 (≈ 180.0% T_dom)
PRBS segment length in samples: [33, 133]
PRBS segment length in seconds: [3.30, 13.30]
     t    MV        CV  CV_clean     noise phase
0  0.0   0.0 -0.037219       0.0 -0.037219  STEP
1  0.1  50.0 -0.468718       0.0 -0.468718  STEP
2  0.2  50.0 -0.115870       0.0 -0.115870  STEP
3  0.3  50.0  0.299440       0.0  0.299440  STEP
4  0.4  50.0 -0.555184       0.0 -0.555184  STEP
phase
PRBS    4600
STEP     400
Name: count, dtype: int64


Identyfikacja obiektu - dekonowulcja w dziedzinie częstotliwości

In [22]:
# ============================================================
# IDENTYFIKACJA DEAD TIME METODĄ DEKONWOLUCJI (FFT)
# u(t) = MV, y(t) = CV_clean
# ============================================================

# 1) bierzemy tylko fragment PRBS (lepsza stacjonarność)
mask_prbs = (df["phase"] == "PRBS")
u = df.loc[mask_prbs, "MV"].values.astype(float)       # wejście
y = df.loc[mask_prbs, "CV_clean"].values.astype(float) # wyjście "bez szumu"

# opcjonalnie: odjęcie średniej (żeby DC nie dominowało w FFT)
u = u - u.mean()
y = y - y.mean()

N_id = len(u)
print(f"Liczba próbek do identyfikacji: {N_id}")

# 2) zero-padding (żeby uniknąć zawijania splotu)
n_fft = 2**int(np.ceil(np.log2(2 * N_id)))   # najbliższa potęga 2 > 2*N
u_pad = np.zeros(n_fft)
y_pad = np.zeros(n_fft)
u_pad[:N_id] = u
y_pad[:N_id] = y

# 3) transformaty Fouriera
U = np.fft.fft(u_pad)
Y = np.fft.fft(y_pad)

# 4) dekonwolucja w dziedzinie częstotliwości:
#    H(ω) = Y(ω) / U(ω) – z lekką regularizacją, żeby nie dzielić przez ~0
eps = 1e-6 * np.max(np.abs(U))
H = Y * np.conj(U) / (np.abs(U)**2 + eps)

# 5) odpowiedź impulsowa: h[n] = IFFT{H(ω)}
h = np.fft.ifft(H).real  # interesuje nas część rzeczywista

# 6) kroimy sensowny zakres odpowiedzi impulsowej
#    (np. pierwsze 400 próbek ≈ 40 s przy dt = 0.1)
n_imp = min(400, len(h))
t_imp = np.arange(n_imp) * dt
h_trunc = h[:n_imp]

# 7) odpowiedź skokowa = całka (suma) z odpowiedzi impulsowej
step_resp = np.cumsum(h_trunc) * dt

# 8) estymacja dead time:
#    zakładamy, że odpowiedź na skok = 0 do chwili L,
#    potem startuje. Bierzemy np. próg 3,5% wartości ustalonej.
g_final = step_resp[-1]             # przybliżona wartość ustalona dla skoku 1
threshold = 0.075 * g_final          # 3,5% poziomu końcowego

idx_dead = np.argmax(step_resp > threshold)
t_dead_est = idx_dead * dt

true_dead = delay_steps * dt

print(f"Estymowany dead time z FFT:  t_dead_est ≈ {t_dead_est:.3f} s")
print(f"Prawdziwy dead time w modelu: t_dead_true ≈ {true_dead:.3f} s")
print(f"Indeks dead time: {idx_dead}, (kroki), vs. {delay_steps} z modelu")

# ============================================================
# WIZUALIZACJA: odpowiedź impulsowa i skokowa z FFT
# ============================================================

fig_imp = go.Figure()
fig_imp.add_trace(go.Scatter(x=t_imp, y=h_trunc,
                             mode="lines", name="h[n] (impulse response)"))
fig_imp.add_vline(x=true_dead,
                  line=dict(dash="dash"),
                  annotation_text="true dead time",
                  annotation_position="top right")
fig_imp.add_vline(x=t_dead_est,
                  line=dict(dash="dot"),
                  annotation_text="est dead time",
                  annotation_position="bottom right")
fig_imp.update_layout(
    title="Odpowiedź impulsowa (z dekonwolucji FFT)",
    xaxis_title="t [s]",
    yaxis_title="h(t)"
)
fig_imp.show()

fig_step = go.Figure()
fig_step.add_trace(go.Scatter(x=t_imp, y=step_resp,
                              mode="lines", name="step response (z FFT)"))
fig_step.add_hline(y=g_final, line=dict(dash="dash"),
                   annotation_text="wartość ustalona (≈ K_proc)",
                   annotation_position="top right")
fig_step.add_hline(y=threshold, line=dict(dash="dot"),
                   annotation_text="próg 2%",
                   annotation_position="bottom right")
fig_step.add_vline(x=true_dead,
                  line=dict(dash="dash"),
                  annotation_text="true dead time",
                  annotation_position="top left")
fig_step.add_vline(x=t_dead_est,
                  line=dict(dash="dot"),
                  annotation_text="est dead time",
                  annotation_position="bottom left")
fig_step.update_layout(
    title="Odpowiedź skokowa (całka z h[n]) + estymacja dead time",
    xaxis_title="t [s]",
    yaxis_title="y_step(t)"
)
fig_step.show()


Liczba próbek do identyfikacji: 4600
Estymowany dead time z FFT:  t_dead_est ≈ 9.100 s
Prawdziwy dead time w modelu: t_dead_true ≈ 12.000 s
Indeks dead time: 91, (kroki), vs. 120 z modelu


In [24]:
import numpy as np

# ============================================================
# KROK 1: HORYZONT PREDYKCJI = DEAD TIME
# ============================================================

h_dead_steps = delay_steps          # horyzont w próbkach
h_dead_time  = h_dead_steps * dt    # horyzont w sekundach

print(f"Horyzont predykcji: {h_dead_steps} próbek ≈ {h_dead_time:.3f} s")

# ============================================================
# KROK 2: SYGNAŁ DOCELOWY (pseudoCV) = CV(t + L)
# ============================================================
# Uwaga: używam CV_clean jako "idealnego" CV bez opóźnienia.
# Jeśli chcesz uczyć sieć na zaszumionych etykietach, zamień na df['CV'].

# 3a) wybór fragmentu do ML – na razie tylko PRBS (bardziej informatywny)
USE_ONLY_PRBS = True

if USE_ONLY_PRBS:
    df_ml = df[df["phase"] == "PRBS"].copy()
else:
    df_ml = df.copy()

df_ml = df_ml.reset_index(drop=True)

# 3b) tworzymy kolumnę CV_target przesuniętą o -h_dead_steps
df_ml["CV_target"] = df_ml["CV_clean"].shift(-h_dead_steps)

# obcinamy końcówkę, gdzie brak etykiet
df_ml = df_ml.iloc[:-h_dead_steps].copy()
df_ml.reset_index(drop=True, inplace=True)

print("df_ml columns:", df_ml.columns)
print(df_ml.head())

# ============================================================
# KROK 3: OKNA CZASOWE POD LSTM
# ============================================================
# Wybieramy cechy wejściowe (na razie: MV + CV z zaszumionego pomiaru)
feature_cols = ["MV", "CV"]   # można później rozszerzyć
target_col   = "CV_target"

window_len = 50  # liczba próbek historii na wejściu

data   = df_ml[feature_cols].values   # shape (N_ml, n_features)
target = df_ml[target_col].values     # shape (N_ml,)

N_ml, n_features = data.shape
print(f"Liczba próbek ML (po obcięciu): {N_ml}, liczba cech: {n_features}")

X_list = []
y_list = []

# dla indeksu t bierzemy okno [t-window_len+1 .. t] jako wejście,
# a etykietą jest CV_target[t] = CV_clean[t + h_dead_steps]
for t_idx in range(window_len - 1, N_ml):
    X_window = data[t_idx - window_len + 1 : t_idx + 1, :]  # (window_len, n_features)
    y_value  = target[t_idx]

    X_list.append(X_window)
    y_list.append(y_value)

X = np.stack(X_list)   # (n_samples, window_len, n_features)
y = np.array(y_list)   # (n_samples,)

print(f"X shape: {X.shape}  (n_samples, window_len, n_features)")
print(f"y shape: {y.shape}  (n_samples,)")

# ============================================================
# KROK 4: PODZIAŁ NA TRAIN / VAL / TEST (CZASOWO)
# ============================================================

n_samples = X.shape[0]
train_end = int(0.6 * n_samples)
val_end   = int(0.8 * n_samples)

X_train, y_train = X[:train_end],        y[:train_end]
X_val,   y_val   = X[train_end:val_end], y[train_end:val_end]
X_test,  y_test  = X[val_end:],          y[val_end:]

print(f"Train samples: {X_train.shape[0]}")
print(f"Val   samples: {X_val.shape[0]}")
print(f"Test  samples: {X_test.shape[0]}")

# ============================================================
# KROK 5: NORMALIZACJA (NA PODSTAWIE TRAIN)
# ============================================================

# spłaszczamy wymiar czasowy, żeby policzyć statystyki po cechach
X_train_flat = X_train.reshape(-1, n_features)

feat_mean = X_train_flat.mean(axis=0)
feat_std  = X_train_flat.std(axis=0) + 1e-8   # żeby nie dzielić przez 0

def normalize_X(X, mean, std):
    return (X - mean) / std

X_train_norm = normalize_X(X_train, feat_mean, feat_std)
X_val_norm   = normalize_X(X_val,   feat_mean, feat_std)
X_test_norm  = normalize_X(X_test,  feat_mean, feat_std)

# (opcjonalnie) normalizacja y – często pomaga w regresji
y_mean = y_train.mean()
y_std  = y_train.std() + 1e-8

y_train_norm = (y_train - y_mean) / y_std
y_val_norm   = (y_val   - y_mean) / y_std
y_test_norm  = (y_test  - y_mean) / y_std

print("feat_mean:", feat_mean)
print("feat_std :", feat_std)
print("y_mean:", y_mean, " y_std:", y_std)

# Na tym etapie masz:
#  - X_train_norm, y_train_norm
#  - X_val_norm,   y_val_norm
#  - X_test_norm,  y_test_norm
# gotowe do podania do modelu LSTM w Keras.


Horyzont predykcji: 120 próbek ≈ 12.000 s
df_ml columns: Index(['t', 'MV', 'CV', 'CV_clean', 'noise', 'phase', 'CV_target'], dtype='object')
      t    MV         CV   CV_clean     noise phase  CV_target
0  40.0  90.0  59.588869  59.865753 -0.276884  PRBS  47.974337
1  40.1  90.0  59.590537  59.761725 -0.171188  PRBS  47.931070
2  40.2  90.0  59.850828  59.656727  0.194101  PRBS  47.897778
3  40.3  90.0  59.551096  59.550791  0.000305  PRBS  47.874351
4  40.4  90.0  59.666682  59.443954  0.222727  PRBS  47.860678
Liczba próbek ML (po obcięciu): 4480, liczba cech: 2
X shape: (4431, 50, 2)  (n_samples, window_len, n_features)
y shape: (4431,)  (n_samples,)
Train samples: 2658
Val   samples: 886
Test  samples: 887
feat_mean: [51.4386757  52.55062749]
feat_std : [39.97411929 14.61447164]
y_mean: 52.34515400851213  y_std: 14.822307038787


In [25]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# ------------------------------------------------------------
# UTRWALENIE LOSOWOŚCI – aby wyniki były powtarzalne
# ------------------------------------------------------------
np.random.seed(42)
tf.random.set_seed(42)

# ------------------------------------------------------------
# PARAMETRY WEJŚCIA SIECI
# window_len     → liczba próbek historii
# n_features     → ile sygnałów na wejściu (MV, CV, itd.)
# ------------------------------------------------------------
n_features = X_train_norm.shape[2]          # np. 2 (MV, CV)
window_len = X_train_norm.shape[1]          # np. 50 kroków historii

# ------------------------------------------------------------
# DEFINICJA MODELU LSTM
# Architektura: LSTM(64) → Dropout → Dense(32) → Dense(1)
# To klasyczna sieć do predykcji jednowymiarowej wartości ciągłej.
# ------------------------------------------------------------
model_lstm = keras.Sequential([

    # wejście ma wymiar (window_len, n_features)
    layers.Input(shape=(window_len, n_features)),

    # LSTM 64 neurony – wychwytuje zależności czasowe
    layers.LSTM(64, return_sequences=False),

    # Dropout zapobiega przeuczeniu – losowo wyłącza neurony
    layers.Dropout(0.2),

    # gęsta warstwa ukryta – dodatkowe przetwarzanie reprezentacji
    layers.Dense(32, activation="relu"),

    # Dropout zapobiega przeuczeniu – losowo wyłącza neurony
    layers.Dropout(0.2),

    # gęsta warstwa ukryta – dodatkowe przetwarzanie reprezentacji
    layers.Dense(16, activation="relu"),

    # wyjście sieci – jedna wartość: przewidywane CV(t+L)
    layers.Dense(1, activation="linear")
])

# ------------------------------------------------------------
# KOMPILACJA MODELU
# optimizer = Adam        → dobrze działa przy danych czasowych
# loss = mse              → klasyczna funkcja dla regresji
# ------------------------------------------------------------
model_lstm.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="mse",
    metrics=["mae"]
)

# wyświetl podsumowanie architektury
model_lstm.summary()

# ------------------------------------------------------------
# CALLBACKS – mechanizmy kontroli treningu
# EarlyStopping: jeśli val_loss nie poprawia się 15 epok → STOP
# ReduceLROnPlateau: gdy utknie → zmniejszamy learning rate
# ------------------------------------------------------------
early_stop = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=15,
    restore_best_weights=True
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,        # zmniejsz LR o 50%
    patience=5,        # po 5 epokach stagnacji
    min_lr=1e-5
)

# ------------------------------------------------------------
# TRENING SIECI
# batch_size 64  → dobry kompromis szybkość/stabilność
# epochs 200     → ale EarlyStopping zatrzyma wcześniej
# ------------------------------------------------------------
history = model_lstm.fit(
    X_train_norm, y_train_norm,
    validation_data=(X_val_norm, y_val_norm),
    epochs=200,
    batch_size=64,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

# ------------------------------------------------------------
# EWALUACJA W SKALI ZNORMALIZOWANEJ
# oceniamy błąd na train, val i test (w normowanej przestrzeni)
# ------------------------------------------------------------
print("\nEVAL – TRAIN:")
model_lstm.evaluate(X_train_norm, y_train_norm, verbose=1)

print("\nEVAL – VAL:")
model_lstm.evaluate(X_val_norm, y_val_norm, verbose=1)

print("\nEVAL – TEST:")
model_lstm.evaluate(X_test_norm, y_test_norm, verbose=1)

# ------------------------------------------------------------
# PREDYKCJE NA ZBIORZE TESTOWYM
# model zwraca wartości w skali znormalizowanej → trzeba odtworzyć skalę fizyczną
# ------------------------------------------------------------
y_test_pred_norm = model_lstm.predict(X_test_norm).flatten()

# odwracamy normalizację: y = y_norm * std + mean
y_test_pred = y_test_pred_norm * y_std + y_mean
y_test_true = y_test   # prawdziwy CV_target w skali fizycznej

# ------------------------------------------------------------
# METRYKI W SKALI FIZYCZNEJ
# MSE, MAE, MAPE – teraz mają sens dla interpretacji procesowej
# ------------------------------------------------------------
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse_test = mean_squared_error(y_test_true, y_test_pred)
mae_test = mean_absolute_error(y_test_true, y_test_pred)

# MAPE trzeba uważać na zera – zabezpieczamy eps
eps = 1e-8
mape_test = np.mean(np.abs((y_test_true - y_test_pred) / (y_test_true + eps))) * 100

print("\n=== METRYKI W SKALI FIZYCZNEJ (TEST) ===")
print(f"MSE  (CV_pred vs CV_target) = {mse_test:.4f}")
print(f"MAE  (CV_pred vs CV_target) = {mae_test:.4f}")
print(f"MAPE (CV_pred vs CV_target) = {mape_test:.2f} %")


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 64)             │        17,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,777 (77.25 KB)

 Trainable params: 19,777 (77.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 1.0288 - mae: 0.8178 - val_loss: 1.1640 - val_mae: 0.9379 - learning_rate: 0.0010
Epoch 2/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.9714 - mae: 0.7841 - val_loss: 1.2301 - val_mae: 0.9555 - learning_rate: 0.0010
Epoch 3/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.9288 - mae: 0.7547 - val_loss: 1.3470 - val_mae: 0.9940 - learning_rate: 0.0010
Epoch 4/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.8656 - mae: 0.7208 - val_loss: 1.3983 - val_mae: 1.0044 - learning_rate: 0.0010
Epoch 5/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.8348 - mae: 0.7063 - val_loss: 1.3329 - val_mae: 0.9839 - learning_rate: 0.0010
Epoch 6/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.8025 - mae: 0.6952 - val_loss: 1.4270 - val_mae: 1.0247 - learning_rate: 0.0010
Epoch 7/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.7630 - mae: 0.6818 - val_loss: 1.3479 - val_mae: 0.9758 - learning_rate: 5.0000e-04
Ep

In [26]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error
import plotly.graph_objects as go

# ------------------------------------------------------------
# 1. PREDYKCJA NA ZBIORZE TESTOWYM (w skali znormalizowanej)
# ------------------------------------------------------------
# model_lstm.predict → zwraca y w skali normowanej (bo tak trenowaliśmy)
y_test_pred_norm = model_lstm.predict(X_test_norm).flatten()   # (n_test,)

# ------------------------------------------------------------
# 2. ODTWORZENIE SKALI FIZYCZNEJ
#    y = y_norm * y_std + y_mean
# ------------------------------------------------------------
y_test_pred = y_test_pred_norm * y_std + y_mean   # predykcja w skali procesu
y_test_true = y_test                              # prawdziwe CV_target(t+L)

# ------------------------------------------------------------
# 3. METRYKI W SKALI FIZYCZNEJ
# ------------------------------------------------------------
mse_test  = mean_squared_error(y_test_true, y_test_pred)
mae_test  = mean_absolute_error(y_test_true, y_test_pred)
eps       = 1e-8
mape_test = np.mean(np.abs((y_test_true - y_test_pred) / (y_test_true + eps))) * 100

print("\n=== TEST – METRYKI W SKALI FIZYCZNEJ ===")
print(f"MSE  (CV_pred vs CV_target) = {mse_test:.4f}")
print(f"MAE  (CV_pred vs CV_target) = {mae_test:.4f}")
print(f"MAPE (CV_pred vs CV_target) = {mape_test:.2f} %")

# ------------------------------------------------------------
# 4. PODGLĄD KILKU PRÓBEK: PRAWDA vs PREDYKCJA
# ------------------------------------------------------------
n_preview = 10  # ile pierwszych próbek z testu pokazać
print("\nPierwsze próbki (TEST):")
for i in range(n_preview):
    print(f"{i:3d}: true = {y_test_true[i]:8.3f}, pred = {y_test_pred[i]:8.3f}, "
          f"err = {y_test_pred[i] - y_test_true[i]:8.3f}")

# ------------------------------------------------------------
# 5. WYKRES W PLOTLY – CV_target vs CV_pred na zbiorze TEST
# ------------------------------------------------------------
idx = np.arange(len(y_test_true))   # indeks próbek testowych (nie czas, ale kolejność)

fig = go.Figure()

# przebieg rzeczywistego CV_target(t+L)
fig.add_trace(go.Scatter(
    x=idx,
    y=y_test_true,
    mode="lines",
    name="CV_target (true)",
))

# przebieg przewidywany przez LSTM
fig.add_trace(go.Scatter(
    x=idx,
    y=y_test_pred,
    mode="lines",
    name="CV_pred (LSTM)",
))

fig.update_layout(
    title="Zbiór TEST – porównanie CV_target vs CV_pred (LSTM)",
    xaxis_title="Indeks próbki (czas testu w kolejności)",
    yaxis_title="CV(t+L)",
)

fig.show()

# ------------------------------------------------------------
# 6. WYKRES BŁĘDU PREDYKCJI NA TEŚCIE
# ------------------------------------------------------------
error_test = y_test_pred - y_test_true

fig_err = go.Figure()
fig_err.add_trace(go.Scatter(
    x=idx,
    y=error_test,
    mode="lines",
    name="error = CV_pred - CV_target",
))
fig_err.update_layout(
    title="Zbiór TEST – błąd predykcji pseudoCV (LSTM)",
    xaxis_title="Indeks próbki",
    yaxis_title="Błąd [jednostki CV]",
)
fig_err.show()


28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step

=== TEST – METRYKI W SKALI FIZYCZNEJ ===
MSE  (CV_pred vs CV_target) = 388.1169
MAE  (CV_pred vs CV_target) = 17.2921
MAPE (CV_pred vs CV_target) = 52.18 %

Pierwsze próbki (TEST):
  0: true =   68.386, pred =   49.631, err =  -18.755
  1: true =   68.564, pred =   49.803, err =  -18.761
  2: true =   68.746, pred =   49.959, err =  -18.787
  3: true =   68.929, pred =   50.150, err =  -18.780
  4: true =   69.116, pred =   50.336, err =  -18.780
  5: true =   69.305, pred =   50.508, err =  -18.797
  6: true =   69.496, pred =   50.653, err =  -18.843
  7: true =   69.690, pred =   50.801, err =  -18.889
  8: true =   69.886, pred =   50.948, err =  -18.938
  9: true =   70.084, pred =   51.060, err =  -19.024
